# Import Statements

In [1]:
import numpy as np
import pandas as pd
import pickle
import gzip
from matplotlib import pyplot as plt
from tqdm import tqdm
import random

# Read Files

In [2]:
def images_file_read(file_name):
    with gzip.open(file_name, 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of images
        image_count = int.from_bytes(f.read(4), 'big')
        # third 4 bytes is the row count
        row_count = int.from_bytes(f.read(4), 'big')
        # fourth 4 bytes is the column count
        column_count = int.from_bytes(f.read(4), 'big')
        # rest is the image pixel data, each pixel is stored as an unsigned byte
        # pixel values are 0 to 255
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8).reshape((image_count, row_count, column_count))
        return images

In [3]:
def labels_file_read(file_name):
    with gzip.open(file_name, 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of labels
        label_count = int.from_bytes(f.read(4), 'big')
        # rest is the label data, each label is stored as unsigned byte
        # label values are 0 to 9
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

In [4]:
train_x = images_file_read('Dataset/mnist/train-images-idx3-ubyte.gz')
print(train_x.shape)

(60000, 28, 28)


In [5]:
train_x = np.reshape(train_x, (60000,784))
print(train_x.shape)

(60000, 784)


In [6]:
train_y = labels_file_read('Dataset/mnist/train-labels-idx1-ubyte.gz')
print(train_y.shape)

(60000,)


In [7]:
test_x = images_file_read('Dataset/mnist/t10k-images-idx3-ubyte.gz')
print(test_x.shape)

(10000, 28, 28)


In [8]:
test_x = np.reshape(test_x, (10000,784))
print(test_x.shape)

(10000, 784)


In [9]:
test_y = labels_file_read('Dataset/mnist/t10k-labels-idx1-ubyte.gz')
print(test_y.shape)

(10000,)


# Preprocessing

In [10]:
# Preprocessing
# train_x = (train_x.astype('float32')/255)
# test_x = (test_x.astype('float32')/255)

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


# Bagging Model

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
def bagging(DTC, train_x, train_y, test_x, test_y, num_of_bags):
    bag_of_models = []
    for i in range(num_of_bags):
        indexes = np.random.randint(low=0, high=train_x.shape[0], size=train_x.shape[0])
        bag_x = [train_x[indexes[i]] for i in range(train_x.shape[0])]
        bag_y = [train_y[indexes[i]] for i in range(train_x.shape[0])]
        DTC.fit(bag_x, bag_y)
        bag_of_models.append(DTC)
    return bag_of_models

In [13]:
# DTC = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, random_state=0)
# bag_of_models = bagging(DTC, train_x, train_y, test_x, test_y, 3)

# Saving Model

In [14]:
# pi_file = open('Q4_pkl_file.pkl', 'wb')
# pickle.dump(bag_of_models, pi_file)
# pi_file.close()

# Loading Model

In [15]:
pi_file = open('Q4_pkl_file.pkl', 'rb')
bag_of_models = pickle.load(pi_file)
pi_file.close()

# Testing

In [16]:
bag_of_predictions = []
for i in range(3):
    bag_of_predictions.append(bag_of_models[i].predict(test_x))

In [17]:
final_prediction = []
for i in range(len(test_y)):
    if bag_of_predictions[1][i] == bag_of_predictions[2][i]:
        final_prediction.append(bag_of_predictions[1][i])
    else:
        if bag_of_predictions[0][i] == bag_of_predictions[1][i] or bag_of_predictions[0][i] == bag_of_predictions[2][i]:
            final_prediction.append(bag_of_predictions[0][i])
        else:
            choice = np.random.randint(0,3)
            final_prediction.append(bag_of_predictions[choice][i])

In [18]:
my_confusion_matrix = np.zeros((10,10), dtype=np.int64)

In [19]:
for i in range(test_y.shape[0]):
    my_confusion_matrix[test_y[i]][final_prediction[i]] += 1

In [20]:
print(my_confusion_matrix)

[[ 912    0    9    4    4   12   16    4   11    8]
 [   0 1109    3    3    1    4    3    2    8    2]
 [  10   13  883   21   13    4   27   30   22    9]
 [   7    7   29  840    6   55    4   15   23   24]
 [   6    7   10    8  842    5   17   12   20   55]
 [  17    5    5   31   11  744   25    2   29   23]
 [  13    6   22    5   26   26  825    4   26    5]
 [   4   12   29   13    4    2    1  932    9   22]
 [   8    8   32   42   24   18   17    8  785   32]
 [   5   10    9   25   43   17    9   23   26  842]]


In [21]:
class_wise_accuracies = np.zeros(10)
for i in range(10):
    class_wise_accuracies[i] = my_confusion_matrix[i][i]/np.sum(my_confusion_matrix[i])
    print('Accuracy of class {}: {}%'.format(i, class_wise_accuracies[i]*100))

print('\nOverall accuracy: {}%'.format(100*np.trace(my_confusion_matrix)/np.sum(my_confusion_matrix, axis=None)))

Accuracy of class 0: 93.06122448979592%
Accuracy of class 1: 97.70925110132158%
Accuracy of class 2: 85.56201550387597%
Accuracy of class 3: 83.16831683168317%
Accuracy of class 4: 85.74338085539715%
Accuracy of class 5: 83.40807174887892%
Accuracy of class 6: 86.1169102296451%
Accuracy of class 7: 90.6614785992218%
Accuracy of class 8: 80.59548254620124%
Accuracy of class 9: 83.44895936570862%

Overall accuracy: 87.14%
